In [1]:
!pip install transformers datasets scikit-learn pandas

In [2]:
import pandas as pd

# Check that your file is loaded
df = pd.read_csv("customersupport_tweets.csv")
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [4]:
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

# Load and preprocess dataset
df = pd.read_csv("customersupport_tweets.csv", parse_dates=["created_at"])

# Separate inbound (customer tweets) and outbound (support replies)
inbound_df = df[df["inbound"] == True].copy()
outbound_df = df[df["inbound"] == False].copy()

# Rename for clarity
inbound_df.rename(columns={"tweet_id": "customer_tweet_id", "created_at": "customer_created_at"}, inplace=True)
outbound_df.rename(columns={"in_response_to_tweet_id": "customer_tweet_id", "created_at": "support_created_at"}, inplace=True)

# Merge to align customer tweet with its response
merged = pd.merge(inbound_df, outbound_df[["customer_tweet_id", "support_created_at"]],
                  on="customer_tweet_id", how="inner")

# Compute response time (in seconds)
merged["response_time_seconds"] = (merged["support_created_at"] - merged["customer_created_at"]).dt.total_seconds()

# Label urgency: urgent = <1 hour
merged["urgency"] = merged["response_time_seconds"].apply(lambda x: 1 if x < 3600 else 0)

# Clean tweet text
def clean_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^A-Za-z\s]', '', text)
    return text.lower().strip()

merged["clean_text"] = merged["text"].astype(str).apply(clean_text)

# Final dataframe
df_final = merged[["clean_text", "urgency"]].dropna()

# Tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

X_train, X_test, y_train, y_test = train_test_split(
    df_final["clean_text"].tolist(),
    df_final["urgency"].tolist(),
    test_size=0.2,
    random_state=42
)

train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            **{key: torch.tensor(val[idx]) for key, val in self.encodings.items()},
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, y_train)
test_dataset = TweetDataset(test_encodings, y_test)

# Model Training
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

# Evaluation
outputs = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(outputs.predictions), dim=1).numpy()

acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds)

print(f"✅ Accuracy: {acc:.4f}")
print(f"✅ F1 Score: {f1:.4f}")

/tmp/ipython-input-4-1377471048.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("customersupport_tweets.csv", parse_dates=["created_at"])
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anisapadukaran (anisapadukaran-manappuram-insurance) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,0.761425
2,No log,0.766662
3,No log,0.788121


✅ Accuracy: 0.3333
✅ F1 Score: 0.5000


In [6]:
import numpy as np # Import numpy
import torch

def predict_urgency(text):
    text_clean = clean_text(text)
    inputs = tokenizer(text_clean, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).numpy()[0]
    label = "Urgent" if np.argmax(probs) == 1 else "Non-Urgent"
    confidence = probs[np.argmax(probs)]
    return label, confidence

# Try a test tweet
test_text = "I’ve been waiting 4 hours with no response. Please help!"
label, confidence = predict_urgency(test_text)

print(f"Prediction: {label} ({confidence:.2%} confidence)")

Prediction: Urgent (66.82% confidence)
